In [1]:
import os
import sys
import pandas as pd
import numpy as np
from Bio import SeqIO, AlignIO
import seaborn as sns
from pathlib import Path
import sys
import atomium
sys.path.append('../../graph_toolbox/')
from feature import calc_named

### Truncate sequences

In [2]:
fasta_input = list(SeqIO.parse('./all_filtered_from_singletons_and_noccds_vs_r751/all_filtered_from_singletons_and_noccds_vs_r751.fasta', 'fasta'))

FileNotFoundError: [Errno 2] No such file or directory: './all_filtered_from_singletons_and_noccds_vs_r751/all_filtered_from_singletons_and_noccds_vs_r751.fasta'

In [ ]:
# for fasta in fasta_input:
#     if len(fasta.seq) > 150:
#         with open("./all_filtered_from_singletons_and_noccds_vs_r751/truncated_filtered_all.fasta", "a+") as handle:
#             SeqIO.write(fasta, handle, "fasta")


### Trim MSA

In [ ]:
def trim_msa(msafile):
    alns = list(AlignIO.parse(msafile, 'fasta'))
    aln = alns[0]
    fs = np.asarray(list(str(aln[0].seq)))
    cols_to_keep = np.where(fs != '-')[0]
    new_aln = np.asarray([list(aln[:,int(c)]) for c in list(cols_to_keep)]).T
#     print(aln)
    seqids = list(SeqIO.parse(msafile, 'fasta'))

    ids = [record.id for record in seqids]
    with open(f"reformated_aln_{msafile.split('/')[-1].split('.')[0]}.aln", 'a') as out_file:
        for _id, alignment in zip(ids, new_aln):
            out_file.write(f'>{_id}\n')
            out_file.write(f"{''.join(alignment)}\n")
trim_msa(msafile = './all_filtered_from_singletons_and_noccds_vs_ra3/mafft_truncated_filtered_all_with_ra3_as_reference.msa')

### Parse al2co output

In [ ]:
def parse_conservation_al2co(consfile: str) -> tuple:
    cons = pd.read_csv(consfile, skipfooter=14, engine='python', delim_whitespace=True, names=['pos', 'aa', 'score', 'gap'])
    cons = cons[cons.aa!='-'] # gaps in the master seq
    cons_seq = "".join(cons.aa.tolist())
    conserved_aa = cons.loc[cons['score'] > 0]
    conserved_aa['residue'] = conserved_aa.aa + conserved_aa.pos.astype(str)
    conserved_aa = conserved_aa.drop(['pos','aa','gap'], axis = 1)
    cols = conserved_aa.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    conserved_aa = conserved_aa[cols]
    conserved_aa = conserved_aa[['residue','score']]
    conserved_aa = dict(zip(conserved_aa['residue'].values, conserved_aa['score']*20))

#         new_cons = []
#         for my_seq_chunk in find_segments(self.data_row.seqres_index, self.data_row.seq):
#             pos = cons_seq.find(my_seq_chunk)
#             assert pos>=0, f'"{my_seq_chunk}" not found in "{cons_seq}"'
#             new_cons.append(cons.iloc[pos:pos+len(my_seq_chunk)])

#         cons = pd.concat(new_cons)
#         cons.reset_index(inplace=True)
#         cons_seq = "".join(cons.aa.tolist())
#         assert cons_seq == self.data_row.seq

#     for key, score in conserved_aa.items():
#         print(key, score)
    return cons_seq, conserved_aa
        
# parse_conservation_al2co('./all_filtered_from_singletons_and_noccds_vs_r751/all_filtered_al2co_conservation.txt')

### Parse pdb with bfactor set for conservation

In [ ]:
def parse_pdb_conservation(pdbfile: str) -> tuple:
    df = PandasPdb().read_pdb(pdbfile)
    df.df['ATOM']['oneletteraa'] = df.amino3to1()['residue_name']
    df.df['ATOM'].ffill(inplace=True)
    onelettersequence = ''.join(df.amino3to1()['residue_name'])
#     print(onelettersequence)
    conserved_aa = df.df['ATOM'].loc[df.df['ATOM']['b_factor'] > 0]
    groupped = conserved_aa.groupby(['residue_number']).max()
    groupped['residue'] = groupped.oneletteraa + groupped.index.astype(str)
    truncated = groupped[['residue','b_factor']]
#     truncated.set_index('residue', inplace=True)
    truncated = dict(zip(truncated['residue'].values, truncated['b_factor']))
    
    return onelettersequence, truncated
parse_pdb_conservation('../md/hth_detached/map/contactmap_mapped_last.pdb')

### Graph toolbox

In [ ]:
ls /home/users/rmadaj

In [5]:
testdf

,disulfide,hydrophobic,cation_pi,arg_arg,salt_bridge,hbond,c,lj,e,cbcb,1/caca,ca_vs_cb,self,is_seq,is_seq_not,is_struct,res1,res2
0,0.0,1.0,1.0,0.0,0.0,0.0,10.000000,10.0,10.000000,0.999993,0.000000,0.0,1.0,0.0,0.0,0.0,0,0
1,0.0,1.0,1.0,0.0,0.0,0.0,10.000000,10.0,10.000000,0.952838,0.262136,0.0,0.0,1.0,0.0,0.0,0,1
2,0.0,1.0,1.0,0.0,0.0,0.0,10.000000,10.0,10.000000,0.995090,0.162715,0.0,0.0,0.0,0.0,1.0,0,2
3,0.0,1.0,0.0,0.0,0.0,0.0,10.000000,10.0,10.000000,1.049483,0.262136,0.0,0.0,1.0,0.0,0.0,1,0
4,0.0,1.0,0.0,0.0,0.0,0.0,10.000000,10.0,10.000000,0.999994,0.000000,0.0,1.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3801,0.0,0.0,0.0,0.0,0.0,0.0,1.302596,0.0,1.302596,1.023786,0.117846,0.0,0.0,0.0,1.0,1.0,305,151
3802,0.0,0.0,0.0,0.0,0.0,0.0,1.302596,0.0,1.302596,1.018463,0.137657,0.0,0.0,0.0,1.0,1.0,305,152
3803,0.0,0.0,0.0,0.0,0.0,0.0,1.302596,0.0,1.302596,1.005952,0.179323,0.0,0.0,0.0,0.0,1.0,305,303
3804,0.0,0.0,0.0,0.0,0.0,0.0,1.302596,0.0,1.302596,1.019030,0.260102,0.0,0.0,1.0,0.0,0.0,305,304


In [3]:
# testfile = '../md/hth_attached/map/test.pdb'
testdf = calc_named(chain = None, pdb_loc = 'test.pdb', t = 9)

def prepare_df_from_toolbox(df: str) -> tuple:
    hth_residues = [x for x in range(0,70)]
    ccd_residues = [x for x in range(70,116)] + [x for x in range (223,265)]
    filtered_df = df[df['res1'].isin(hth_residues)]
    filtered_df = filtered_df[filtered_df['res2'].isin(ccd_residues)]
#     filtered_df = filtered_df.iloc[:, [0,1,2,3,4,5, -2,-1]]
    groupped = filtered_df.groupby(['res1','res2']).max()
    return groupped

    
    
prepare_df_from_toolbox(testdf)

disulfide  hydrophobic  cation_pi  arg_arg  salt_bridge  hbond  \
res1 res2                                                                   
23   109         0.0          0.0        0.0      0.0          0.0    0.0   
34   109         0.0          0.0        0.0      0.0          0.0    0.0   
     110         0.0          0.0        0.0      0.0          0.0    0.0   
     112         0.0          0.0        0.0      0.0          0.0    0.0   
     113         0.0          0.0        0.0      0.0          0.0    0.0   
...              ...          ...        ...      ...          ...    ...   
68   75          0.0          0.0        0.0      0.0          0.0    0.0   
69   70          0.0          0.0        0.0      0.0          0.0    0.0   
     71          0.0          0.0        0.0      0.0          0.0    0.0   
     72          0.0          0.0        0.0      0.0          0.0    0.0   
     73          0.0          0.0        0.0      0.0          0.0    0.0   

                  c   lj         e          cbcb    1/caca  ca_vs_cb  self  \
res1 res2                                                                    
23   109   7.773123  0.0  7.773123  1.066441e+00  0.118006       1.0   0.0   
34   109   3.519099  0.0  3.519099  1.034129e+00  0.120580       1.0   0.0   
     110   3.519099  0.0  3.519099  1.000982e+00  0.115377       0.0   0.0   
     112   3.519099  0.0  3.519099  9.927235e-01  0.116776       0.0   0.0   
     113   3.519099  0.0  3.519099  1.019112e+00  0.161034       1.0   0.0   
...             ...  ...       ...           ...       ...       ...   ...   
68   75    3.189008  0.0  3.189008  9.939362e-01  0.118827       1.0   0.0   
69   70    1.232562  0.0  1.232562  1.036114e+00  0.256699       0.0   0.0   
     71    1.232562  0.0  1.232562  1.070033e+00  0.177750       0.0   0.0   
     72    1.232562  0.0  1.232562  1.000000e-20  0.201023       0.0   0.0   
     73    1.232562  0.0  1.232562  1.000000e-20  0.153757       0.0   0.0   

           is_seq  is_seq_not  is_struct  
res1 res2                                 
23   109      0.0         1.0        1.0  
34   109      0.0         1.0        1.0  
     110      0.0         1.0        1.0  
     112      0.0         1.0        1.0  
     113      0.0         1.0        1.0  
...           ...         ...        ...  
68   75       0.0         1.0        1.0  
69   70       1.0         0.0        0.0  
     71       0.0         0.0        1.0  
     72       0.0         0.0        1.0  
     73       0.0         0.0        1.0  

[77 rows x 16 columns]

### Compare pdb and al2co

In [ ]:
def compare_conservations(al2coseq: str, al2co: dict, pdbseq:str, pdb: dict):
#     print(type(al2co))
        conservations_dict = {}
        for pdbresids, pdbcons in pdb.items():
            for alresids, alcons in al2co.items():
                if alresids in pdbresids:
                    new_dict = {alresids: [pdbcons, alcons]}
                    conservations_dict.update(new_dict)
        return conservations_dict


    
al2cofile = './all_filtered_from_singletons_and_noccds_vs_r751/all_filtered_al2co_conservation.txt'
pdbfile = '../md/r751-dna/map/contactmap_mapped.pdb'

al2coseq, al2co = parse_conservation_al2co(al2cofile)
pdbseq, pdb = parse_pdb_conservation(pdbfile)

dict_df = pd.DataFrame.from_dict(compare_conservations(al2coseq, al2co, pdbseq, pdb))
dict_df = dict_df.rename(index={0: "b-factor (0-100)", 1: "al2co, 0-5"})
print(dict_df)

### Create bigger df from *compare_conservations*

In [ ]:
ultimate_df = pd.DataFrame()
for path in Path('../md/').rglob('*mapped*.pdb'):
    pdbseq, pdb = parse_pdb_conservation(str(path))
    dict_df = pd.DataFrame.from_dict(compare_conservations(al2coseq, al2co, pdbseq, pdb))
    dict_df = dict_df.rename(index={0: "contacts score", 1: "conservation score"})
#     dict_df['source'] = str(path).split('/')[-3]
    dict_df.insert(0, 'source', str(path).split('/')[-3])
    ultimate_df = ultimate_df.append(dict_df)
ultimate_df

### Kamil toolbox

### CCMpred visualisation

In [ ]:
def visualise_ccmpred(ccmpredmat: str):
    ar = np.genfromtxt(ccmpredmat, delimiter='\t')
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    sns.heatmap(ar, cmap='coolwarm')
visualise_ccmpred('./all_filtered_from_singletons_and_noccds_vs_r751/ccmpred.out')

### DSSP if needed

In [ ]:
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
dssp_dict, keys = dssp_dict_from_pdb_file('./all_filtered_from_singletons_and_noccds_vs_r751/contactmap_mapped_attached.pdb')
acceptable_ss = ['E', 'H']
sse_residues = ""
for key in keys:
    residue = key[1][1]
    ss = dssp_dict[key][1]
    if ss in acceptable_ss:
        sse_residues += ":"
        sse_residues += str(residue)
        sse_residues += ","
print(sse_residues)

In [ ]:
import re
file = open('kamil.pdb')
for line in file:
    if line.startswith('HET'):
        print(line)